In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Auto-Scores-National-Multiple-Choice-Test-master")
import imutils
import numpy as np
import cv2
from math import ceil
from model import CNN_Model
from collections import defaultdict
from google.colab.patches import cv2_imshow
import pandas as pd
#from openpyxl import load_workbook
import math
from xlwt import Workbook
wb = Workbook()
sheet1 = wb.add_sheet('sheet 1')
sheet1.write(0, 0, 'STT')
sheet1.write(0, 1, 'Số báo danh')
sheet1.write(0, 2, 'Mã đề thi')
sheet1.write(0, 3, 'Kết quả')

In [ ]:
def get_x(s):
    return s[1][0]
def get_y(s):
    return s[1][1]
def get_h(s):
    return s[1][3]
def get_x_ver1(s):
    s = cv2.boundingRect(s)
    return s[0] * s[1]

In [ ]:
def crop_image(img):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #cv2_imshow(gray_img)
    blurred = cv2.GaussianBlur(gray_img, (5, 5), 0)
    #cv2_imshow(blurred)
    img_canny = cv2.Canny(blurred, 100, 200)
    #cv2_imshow(img_canny)

    # find contours
    cnts = cv2.findContours(img_canny.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    ans_blocks = []
    x_old, y_old, w_old, h_old = 0, 0, 0, 0

    # ensure that at least one contour was found
    if len(cnts) > 0:
        # sort the contours according to their size in descending order
        cnts = sorted(cnts, key=get_x_ver1)

        # loop over the sorted contours
        for i, c in enumerate(cnts):
            x_curr, y_curr, w_curr, h_curr = cv2.boundingRect(c)

            if w_curr * h_curr > 100000:
                # check overlap contours
                check_xy_min = x_curr * y_curr - x_old * y_old
                check_xy_max = (x_curr + w_curr) * (y_curr + h_curr) - (x_old + w_old) * (y_old + h_old)

                # if list answer box is empty
                if len(ans_blocks) == 0:
                    ans_blocks.append(
                        (gray_img[y_curr:y_curr + h_curr, x_curr:x_curr + w_curr], [x_curr, y_curr, w_curr, h_curr]))
                    # update coordinates (x, y) and (height, width) of added contours
                    x_old = x_curr
                    y_old = y_curr
                    w_old = w_curr
                    h_old = h_curr
                elif check_xy_min > 20000 and check_xy_max > 20000:
                    ans_blocks.append(
                        (gray_img[y_curr:y_curr + h_curr, x_curr:x_curr + w_curr], [x_curr, y_curr, w_curr, h_curr]))
                    # update coordinates (x, y) and (height, width) of added contours
                    x_old = x_curr
                    y_old = y_curr
                    w_old = w_curr
                    h_old = h_curr

        # sort ans_blocks according to x coordinate
        sorted_ans_blocks = sorted(ans_blocks, key=get_x)
        return sorted_ans_blocks

In [ ]:
def process_ans_blocks(ans_blocks):
    list_answers = []
    # Loop over each block ans in
    for ans_block in ans_blocks:
        ans_block_img = np.array(ans_block[0])
        offset1 = ceil(ans_block_img.shape[0] / 6)
        # Loop over each box in answer block
        for i in range(6):
            box_img = np.array(ans_block_img[i * offset1:(i + 1) * offset1, :])
            height_box = box_img.shape[0]
            
            box_img = box_img[14:height_box - 14, :]
            offset2 = ceil(box_img.shape[0] / 5)

            # loop over each line in a box
            for j in range(5):
                list_answers.append(box_img[j * offset2:(j + 1) * offset2, :])

    return list_answers

In [ ]:
def process_list_ans(list_answers):
    list_choices = []
    offset = 44
    start = 32

    for answer_img in list_answers:
        for i in range(4):
            bubble_choice = answer_img[:, start + i * offset:start + (i + 1) * offset]
            bubble_choice = cv2.threshold(bubble_choice, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            bubble_choice = cv2.resize(bubble_choice, (28, 28), cv2.INTER_AREA)
            #cv2_imshow(bubble_choice)
            bubble_choice = bubble_choice.reshape((28, 28, 1))
            list_choices.append(bubble_choice)

    if len(list_choices) != 480:
        raise ValueError("Length of list_choices must be 480")
    return list_choices

In [ ]:
def map_answer(idx):
    if idx % 4 == 0:
        answer_circle = "A"
    elif idx % 4 == 1:
        answer_circle = "B"
    elif idx % 4 == 2:
        answer_circle = "C"
    else:
        answer_circle = "D"
    return answer_circle

In [ ]:

def drawText(image, txt):
	# font
	font = cv2.FONT_HERSHEY_SIMPLEX
	# org
	org = (50, 50)
	# fontScale
	fontScale = 1
	# Blue color in BGR
	color = (130, 130, 0)
	# Line thickness of 2 px
	thickness = 2
	# Using cv2.putText() method
	image = cv2.putText(image, txt, org, font,
						fontScale, color, thickness, cv2.LINE_AA)
	return

# Ham tinh khoang cach giua hai deim
def distance(p1,p2):
    my_dist = math.sqrt(((p1[0] - p2[0]) ** 2) + ((p1[1] - p2[1]) ** 2))
    return  my_dist

def sort_contours(cnts, method="left-to-right"):
	# initialize the reverse flag and sort index
	reverse = False
	i = 0
	# handle if we need to sort in reverse
	if method == "right-to-left" or method == "bottom-to-top":
		reverse = True
	# handle if we are sorting against the y-coordinate rather than
	# the x-coordinate of the bounding box
	if method == "top-to-bottom" or method == "bottom-to-top":
		i = 1
	# construct the list of bounding boxes and sort them from top to
	# bottom
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
	(cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
		key=lambda b:b[1][i], reverse=reverse))
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)

def order_points(pts):

	rect = np.zeros((4, 2), dtype="float32")

	s = pts.sum(axis=1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	diff = np.diff(pts, axis=1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]

	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):


	rect = order_points(pts)
	(tl, tr, br, bl) = rect

	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped

def find_corner_by_rotated_rect(box,approx):
    corner = []
    for p_box in box:
        min_dist = 999999999
        min_p = None
        for p in approx:
            dist = distance(p_box, p[0])
            if dist < min_dist:
                min_dist = dist
                min_p = p[0]
        corner.append(min_p)

    corner = np.array(corner)
    return corner


In [ ]:
def find_area(input,image) :
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Làm mờ ảnh bằng hàm GaussianBlur giảm nhiễu
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    #cv2.imshow('cam', blurred)
    #cv2.waitKey(30)
    # Threshol lấy ngưỡng thích ứng và độ giãn nở để hiển thị các tính năng chính của hình ảnh
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    #cv2.imshow("Anh nhị phân ", thresh)
    #cv2.waitKey()
    # 3. Tim khung ben ngoai de tach van ban khoi nen
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    # sắp xếp các counter theo diện tích
    contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)
    approx = cv2.approxPolyDP(contours[input], 0.008 * cv2.arcLength(contours[input], True), True)
    rect = cv2.minAreaRect(contours[input])
    box = cv2.boxPoints(rect)

    #Thuc hien transform de xoay van ban
    corner = find_corner_by_rotated_rect(box,approx)
    image = four_point_transform(image,corner)
    wrap = four_point_transform(thresh,corner)

    #resize vùng báo danh
    width = int(image.shape[1] * 2)
    height = int(image.shape[0] * 2)
    dim = (width, height)
    image2=cv2.resize(image.copy(),dim,interpolation=cv2.INTER_AREA)
    gray_img = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    # remove noise by blur image and threshold
    blurred_2 = cv2.GaussianBlur(gray_img, (9, 9), 0)
    thresh_2 = cv2.adaptiveThreshold(blurred_2, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    #trả về ảnh nhị phân vùng số báo danh sau khi phóng to
    return thresh_2

In [ ]:
def findcorrect(thresh_2):
    contours = cv2.findContours(thresh_2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    tickcontours = []
    for c in contours:
        approx1 = cv2.approxPolyDP(c, 0.001 * cv2.arcLength(c, True), True)
        (x, y, w, h) = cv2.boundingRect(c)
        ar = w / float(h)
        r= w/2
        area = cv2.contourArea(c)
        if w >= 29 and h >= 28 and 0.9 <= ar <= 1.1 and len(approx1) > 7:
            tickcontours.append(c)
            #print(w, h)
    tickcontours = sort_contours(tickcontours, method="left-to-right")[0]
    correct = '' # bien luu so bao danh
    a=0
    count = 0 # xac dinh vi tri to mau tren moi cot
    #print(len(tickcontours))
    for (q, i) in enumerate(np.arange(0, len(tickcontours), 10)):

        # Dinh nghia mau rieng cho tung cot
        color = (100,a,1)
        a+=50
        # Sap xep cac contour theo cot
        cnts = sort_contours(tickcontours[i:i + 10], method="top-to-bottom")[0]
        #cv2.drawContours(image2, cnts, -1, color, 3)
        #print(q, i)
        choice = None
        total = 0
        # Duyet qua cac contour trong cot
        for (j, c) in enumerate(cnts):
            #print ("j=",j)
            # Tao mask de xem muc do to mau cua contour
            mask = np.zeros(thresh_2.shape, dtype= "uint8")
            cv2.drawContours(mask, [c],-1 , 255, -1)
            mask = cv2.bitwise_and(thresh_2, thresh_2, mask= mask)
            total = cv2.countNonZero(mask)
            #print(total)
            # Lap de chon contour to mau dam nhat
            if choice is None or total > choice[0]:
               choice = (total, j)
               # Neu dung Thi to mau xanh
               count = j
               color = (0, 255, 0)
        correct+=str(count)
    return correct

In [ ]:
right_anser1 = {1: ['B'], 2: ['D'], 3: ['A'], 4: ['A'], 5: ['A'], 6: ['A'], 7: ['A'], 8: ['A'], 9: ['C'], 10: ['B'], 
                   11: ['A'], 12: ['C'], 13: ['A'], 14: ['B'], 15: ['A'], 16: ['C'], 17: ['B'], 18: ['A'], 19: ['C'], 20: ['B'], 
                   21: ['D'], 22: ['D'], 23: ['B'], 24: ['C'], 25: ['C'], 26: ['C'], 27: ['B'], 28: ['A'], 29: ['B'], 30: ['D'], 
                   31: ['C'], 32: ['B'], 33: ['C'], 34: ['B'], 35: ['C'], 36: ['D'], 37: ['C'], 38: ['D'], 39: ['A'], 40: ['C'], 
                   41: ['A'], 42: ['C'], 43: ['C'], 44: ['C'], 45: ['C'], 46: ['D'], 47: ['B'], 48: ['C'], 49: ['C'], 50: ['C'], 
                   51: ['B'], 52: ['C'], 53: ['A'], 54: ['C'], 55: ['C'], 56: ['A'], 57: ['B'], 58: ['A'], 59: ['D'], 60: ['D'],
                   51: ['B'], 52: ['C'], 53: ['A'], 54: ['C'], 55: ['C'], 56: ['A'], 57: ['B'], 58: ['A'], 59: ['D'], 60: ['D'],
                   61: ['B'], 62: ['C'], 63: ['A'], 64: ['C'], 65: ['C'], 66: ['A'], 67: ['B'], 68: ['A'], 69: ['D'], 70: ['D'],
                   71: ['B'], 72: ['C'], 73: ['A'], 74: ['C'], 75: ['C'], 76: ['A'], 77: ['B'], 78: ['A'], 79: ['D'], 80: ['D'],
                   81: ['B'], 82: ['C'], 83: ['A'], 84: ['C'], 85: ['C'], 86: ['A'], 87: ['B'], 88: ['A'], 89: ['D'], 90: ['D'],
                   91: ['B'], 92: ['C'], 93: ['A'], 94: ['C'], 95: ['C'], 96: ['A'], 97: ['B'], 98: ['A'], 99: ['D'], 100: ['D'],
                   101: ['A'], 102: ['C'], 103: ['D'], 104: ['B'], 105: ['A'], 106: ['C'], 107: ['B'], 108: ['A'], 109: ['C'], 110: ['B'], 
                   111: ['D'], 112: ['D'], 113: ['B'], 114: ['C'], 115: ['C'], 116: ['C'], 117: ['B'], 118: ['A'], 119: ['B'], 120: ['D']}
right_anser2 = {1: ['B'], 2: ['D'], 3: ['A'], 4: ['A'], 5: ['C'], 6: ['B'], 7: ['A'], 8: ['D'], 9: ['C'], 10: ['B'], 
                   11: ['A'], 12: ['C'], 13: ['D'], 14: ['B'], 15: ['A'], 16: ['C'], 17: ['B'], 18: ['A'], 19: ['C'], 20: ['B'], 
                   21: ['D'], 22: ['D'], 23: ['B'], 24: ['C'], 25: ['C'], 26: ['C'], 27: ['B'], 28: ['A'], 29: ['B'], 30: ['D'], 
                   31: ['C'], 32: ['B'], 33: ['C'], 34: ['B'], 35: ['C'], 36: ['D'], 37: ['C'], 38: ['D'], 39: ['A'], 40: ['C'], 
                   41: ['A'], 42: ['C'], 43: ['C'], 44: ['C'], 45: ['C'], 46: ['D'], 47: ['B'], 48: ['C'], 49: ['C'], 50: ['C'], 
                   51: ['B'], 52: ['C'], 53: ['A'], 54: ['C'], 55: ['C'], 56: ['A'], 57: ['B'], 58: ['A'], 59: ['D'], 60: ['D'],
                   51: ['B'], 52: ['C'], 53: ['A'], 54: ['C'], 55: ['C'], 56: ['A'], 57: ['B'], 58: ['A'], 59: ['D'], 60: ['D'],
                   61: ['B'], 62: ['C'], 63: ['A'], 64: ['C'], 65: ['C'], 66: ['A'], 67: ['B'], 68: ['A'], 69: ['D'], 70: ['D'],
                   71: ['B'], 72: ['C'], 73: ['A'], 74: ['C'], 75: ['C'], 76: ['A'], 77: ['B'], 78: ['A'], 79: ['D'], 80: ['D'],
                   81: ['B'], 82: ['C'], 83: ['A'], 84: ['C'], 85: ['C'], 86: ['A'], 87: ['B'], 88: ['A'], 89: ['D'], 90: ['D'],
                   91: ['B'], 92: ['C'], 93: ['A'], 94: ['C'], 95: ['C'], 96: ['A'], 97: ['B'], 98: ['A'], 99: ['D'], 100: ['D'],
                   101: ['A'], 102: ['C'], 103: ['D'], 104: ['B'], 105: ['A'], 106: ['C'], 107: ['B'], 108: ['A'], 109: ['C'], 110: ['B'], 
                   111: ['D'], 112: ['D'], 113: ['B'], 114: ['C'], 115: ['C'], 116: ['C'], 117: ['B'], 118: ['A'], 119: ['B'], 120: ['D']}  
#Df1 = pd.DataFrame({'SBD':[sobaodanh],'MaDe':[madethi],'Diem':[correctt]})
#Df1.to_excel("/content/ket_qua.xlsx")
row = 1
col = 0
sheet1.col(1).width = 4000

In [ ]:
for k in range(1,5) :
  img = cv2.imread("/content/tệp_phiếu_thi/phieu"+str(k)+".jpg")
  image = img.copy()
  #print("",img.shape)
  thresh_MDT = find_area(34,image)
  thresh_SBD = find_area(7,image)
  madethi= findcorrect(thresh_MDT)
  sobaodanh= findcorrect(thresh_SBD)
  list_ans_boxes = crop_image(img)
  list_ans = process_ans_blocks(list_ans_boxes)
  list_ans = process_list_ans(list_ans)
  results = defaultdict(list)
  model = CNN_Model('/content/drive/MyDrive/Auto-Scores-National-Multiple-Choice-Test-master/weight.h5').build_model(rt=True)
  results = defaultdict(list)
  list_ans = np.array(list_ans)
  scores = model.predict_on_batch(list_ans / 255.0)
  correctt = 0

  if madethi == "001":
      right_anser = right_anser1
  if madethi == "011":
      right_anser = right_anser2
  for idx, score in enumerate(scores):
      question = idx // 4
      # score [unchoiced_cf, choiced_cf]
      if score[1] > 0.9:  # choiced confidence score > 0.9
          chosed_answer = map_answer(idx)
          results[question + 1].append(chosed_answer)
          if right_anser[question+1] == results[question+1]:
            correctt+=1
  print("phieu"+str(k) )         
  print("Dap an cua thi sinh la: ",results)
  print("So Cau Dung la: ",correctt)
  print("Mã đề thi : ",madethi)
  print("Số báo danh : ",sobaodanh)
  sheet1.write(row, col, k)
  sheet1.write(row, col + 1,sobaodanh)
  sheet1.write(row, col + 2,madethi)
  sheet1.write(row, col + 3,correctt)
  row += 1

wb.save("/content/ket_qua1.xls") 



phieu1
Dap an cua thi sinh la:  defaultdict(<class 'list'>, {1: ['B'], 2: ['D'], 3: ['B', 'C', 'D'], 4: ['A'], 5: ['C'], 6: ['B'], 7: ['A'], 8: ['D'], 9: ['C'], 10: ['B'], 11: ['A'], 12: ['C'], 13: ['D'], 14: ['B'], 15: ['A'], 16: ['C'], 17: ['B'], 18: ['A'], 19: ['C'], 20: ['B'], 21: ['D'], 23: ['B'], 24: ['C'], 26: ['C'], 27: ['B'], 28: ['A'], 29: ['B'], 30: ['D'], 32: ['B'], 34: ['B'], 36: ['D'], 38: ['D'], 39: ['A'], 41: ['A'], 46: ['D'], 47: ['B'], 51: ['B'], 53: ['A'], 56: ['A'], 58: ['B'], 59: ['B'], 60: ['B'], 66: ['A'], 70: ['D'], 73: ['D']})
So Cau Dung la:  36
Mã đề thi :  001
Số báo danh :  112522
phieu2
Dap an cua thi sinh la:  defaultdict(<class 'list'>, {1: ['B'], 2: ['D'], 3: ['B', 'C', 'D'], 4: ['A'], 5: ['C'], 6: ['B'], 7: ['A'], 8: ['D'], 9: ['C'], 10: ['B'], 11: ['A'], 12: ['C'], 13: ['D'], 14: ['B'], 15: ['A'], 16: ['C'], 17: ['B'], 18: ['A'], 19: ['C'], 20: ['B'], 21: ['D'], 23: ['B'], 24: ['C'], 26: ['C'], 27: ['B'], 28: ['A'], 29: ['B'], 30: ['D'], 32: ['B'], 33

In [ ]:
#Df1 = pd.DataFrame({'SBD':[sobaodanh],'MaDe':[madethi],'Diem':[correctt]})
#Df1.to_excel("output.xlsx")

In [ ]:
#df1 = pd.DataFrame({'lkey': ['SBD', 'MaDe', 'Diem'],
#                    'value': [sobaodanh, madethi, correctt]})
#df1.to_excel("output.xlsx")

In [ ]:
#with pd.ExcelWriter('/content/output.xlsx',
#                    mode= ) as writer: 
#    Df2.to_excel(writer,sheet_name='rsi')

In [ ]:
#Df2 = pd.DataFrame({'SBD':[sobaodanh],'MaDe':[madethi],'Diem':[correctt]})
#frames = [Df1, Df2]
#Df1 = pd.concat(frames)
#Df1.to_excel("output.xlsx")

In [ ]:
#Df2 = pd.DataFrame({'SBD':[sobaodanh],'MaDe':[madethi],'Diem':[correct]})
#Df1.merge(Df2, left_on='lkey', right_on='rkey')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
